In [ ]:
# !git clone https://github.com/fluentpython/example-code-2e.git
# %cd /content/example-code-2e/05-data-classes

Cloning into 'example-code-2e'...
remote: Enumerating objects: 3524, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 3524 (delta 104), reused 94 (delta 94), pack-reused 3393
Receiving objects: 100% (3524/3524), 13.38 MiB | 32.87 MiB/s, done.
Resolving deltas: 100% (1911/1911), done.
/content/example-code-2e/05-data-classes


# Data Class Builders

Data classes, introduced in `Python 3.7` via `PEP 557,` are decorator and functions for creating classes primarily to store data.

* Various class builders serve as shortcuts to write data classes:

1. `collections.namedtuple`: A straightforward method to create data classes. Available since Python 2.6.
2.  `typing.NamedTuple`: Introduced in Python 3.5; class syntax added in 3.6. Requires type hints on the fields.
3. `@dataclasses.dataclass`: A class decorator offering more customization than other methods. Available from Python 3.7 onwards. Provides various options, potentially adding complexity.

**NOTE**: `typing.TypedDict`: While it might appear similar to other data class builders, it's distinct.
Found after typing.NamedTuple in Python 3.9's typing module documentation. It does not create concrete classes for instantiation. It provides syntax for type hints for function parameters and variables accepting mapping values used as records.

## Overview of Data Class Builders

In [ ]:
class Coordinate:
    def __init__(self, lat, lon): # latitude and longitude attributes
        self.lat = lat
        self.lon = lon

In [ ]:
moscow = Coordinate(55.76, 37.62)
moscow  # __repr__ inherited from object is not very helpful.

In [ ]:
location = Coordinate(55.76, 37.62)
location

In [ ]:
Coordinate.__doc__

In [ ]:
location == moscow # Meaningless ==; the __eq__ method inherited from object compares object IDs.

False

In [ ]:
# Comparing two coordinates requires explicit comparison of each attribute.
print((location.lat, location.lon) == (moscow.lat, moscow.lon))

True


**Data Class Builders** in Python:
* Automatically provide essential methods:
 * `__init__`
 * `__repr__`
 * `__eq__`

* They also come with other handy features.
 * Key Points: No dependence on inheritance for their operation.
 * `collections.namedtuple` and `typing.NamedTuple` create classes that subclass tuple.
 * `@dataclass` is a class decorator with no impact on the class hierarchy.
Various metaprogramming techniques are employed to inject methods and attributes into the class.

### `namedtuple`

- `namedtuple` is a factory function that builds a subclass of tuple with the name and fields you specif

In [ ]:
from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat lon')
print(Coordinate)
print(Coordinate.__doc__) # docstring for the new class

<class '__main__.Coordinate'>
Coordinate(lat, lon)


In [ ]:
print(issubclass(Coordinate, tuple))

True


In [ ]:
moscow = Coordinate(55.756, 37.617) # Useful __repr__.
print(moscow)

Coordinate(lat=55.756, lon=37.617)


In [ ]:
print(moscow == Coordinate(lat=55.756, lon=37.617)) # Meaningful __eq__.

True


### `typing.NamedTuple`
NamedTuple provides the same functionality, adding a type annotation to each field

* NOTE: using type hints with NamedTuple doesn't enforce type-checking at runtime; it's primarily for static type analysis tools like mypy or enhancing code clarity for developers.

In [ ]:
import typing

# typing.NamedTuple with type hints:
class Coordinate(typing.NamedTuple):
    lat: float
    lon: float

In [ ]:
# typing.NamedTuple using keyword arguments:

# Coordinate = typing.NamedTuple('Coordinate',[('lat', float), ('lon', float)])
Coordinate = typing.NamedTuple('Coordinate',lat=float,lon=float)  # more readable

In [ ]:
print(issubclass(Coordinate, tuple))
print(typing.get_type_hints(Coordinate))

True
{'lat': <class 'float'>, 'lon': <class 'float'>}


In [ ]:
# Example 5-2: Using typing.NamedTuple with a Class Statement
from typing import NamedTuple

class Coordinate(NamedTuple):  # Coordinate not inherits from the NamedTuple class
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        ew = 'E' if self.lon >= 0 else 'W'
        return f"{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{ew}"


NOTE: The `typing.NamedTuple` is indeed special and doesn't behave like a typical base class in traditional inheritance.

It makes use of `metaclasses` to achieve its behavior, which is an advanced topic in Python.

In [ ]:
print(Coordinate.mro()) # Coordinate directly inherits from tuple, not from NamedTuple

[<class '__main__.Coordinate'>, <class 'tuple'>, <class 'object'>]


In [ ]:
issubclass(Coordinate, NamedTuple)

TypeError: ignored

In [ ]:
issubclass(Coordinate, tuple)

True

### @dataclass

- **Definition**: A decorator to auto-generate special methods for classes, like `__init__`, `__repr__`, and `__eq__`.

- **Module**: Part of the `dataclasses` module introduced in Python 3.7.

- **Field Defaults**: Supports default values for fields. If no default is provided, the field is assumed to be required in the `__init__` method.

- **Type Hints**: Encourages the use of type hints, but doesn't enforce them at runtime.

- **Immutability**: Create immutable instances using `@dataclass(frozen=True)`.

- **Comparison**
  - Auto-generates comparison (`__eq__`) method. Can also auto-generate ordering methods (`__lt__`, `__le__`, `__gt__`, `__ge__`) with the `order=True` parameter.

- **Post-Initialization**: Supports a `__post_init__` method that runs after the generated `__init__` method.

- **Field Metadata**: Allows for field metadata which can be used for various purposes, like validation or serialization.

- **Inheritance**: Compatible with class inheritance. Base classes can be dataclasses as well.

- **Utility Functions**
  - The `dataclasses` module provides utility functions like `asdict`, `astuple`, and `replace` for working with dataclass instances.



In [ ]:
# Example 5-3. dataclass/coordinates.py
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self) -> str:
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

coord = Coordinate(55.8, 37.6)
coord

Coordinate(lat=55.8, lon=37.6)

## Main Features

| Feature                        | `namedtuple`   | `NamedTuple`    | `dataclass`                                     |
|--------------------------------|----------------|-----------------|-------------------------------------------------|
| Mutable instances              | NO             | NO              | YES                                             |
| Class statement syntax         | NO             | YES             | YES                                             |
| Construct dict                 | `x._asdict()`  | `x._asdict()`   | `dataclasses.asdict(x)`                         |
| Get field names                | `x._fields`    | `x._fields`     | `[f.name for f in dataclasses.fields(x)]`       |
| Get defaults                   | `x._field_defaults` | `x._field_defaults` | `[f.default for f in dataclasses.fields(x)]` |
| Get field types                | N/A            | `x.__annotations__` | `x.__annotations__`                           |
| New instance with changes      | `x._replace(...)` | `x._replace(...)` | `dataclasses.replace(x, ...)`              |
| New class at runtime           | `namedtuple(...)` | `NamedTuple(...)`  | `dataclasses.make_dataclass(...)`         |


## Key Takeaways on Data Class Builders:

- **Mutability**:
  - `collections.namedtuple` and `typing.NamedTuple` create **immutable** tuple subclasses.
  - `@dataclass` produces **mutable** classes by default.
  - Use `frozen=True` with `@dataclass` to make it immutable, preventing post-initialization assignment.

- **Class Syntax**:
  - Both `typing.NamedTuple` and `@dataclass` support the standard class statement, allowing for easier method addition and docstrings.

- **Dictionary Construction**:
  - `namedtuple` and `NamedTuple`: Use `._asdict()` method.
  - `@dataclass`: Use `dataclasses.asdict()` function.

- **Field Metadata**:
  - Retrieve field names and defaults:
    - `namedtuple`: `._fields` and `._fields_defaults`.
    - `@dataclass`: Use `dataclasses.fields()` function.
  - Field types:
    - `typing.NamedTuple` and `@dataclass` use `__annotations__` for type hints.
    - Prefer `typing.get_type_hints()` over direct `__annotations__` access.

- **Instance Modification**:
  - Create a new modified instance:
    - `namedtuple`: Use `x._replace(**kwargs)`.
    - `@dataclass`: Use `dataclasses.replace(x, **kwargs)`.

- **Runtime Class Creation**:
  - For dynamic class creation:
    - Use standard function call syntax for `collections.namedtuple` and `typing.NamedTuple`.
    - `@dataclass` provides `dataclasses.make_dataclass()` for this purpose.


## Classic Named Tuples

In [ ]:
# Example 5-4. Defining and using a named tuple type
from collections import namedtuple

# Two parameters are required to create a named tuple: a class name and a list of field names
# As a tuple subclass, City inherits useful methods such as __eq__ and __lt__
City = namedtuple('City', 'name country population coordinates')

tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [ ]:
tokyo.population  # access the fields by name or position.

36.933

In [ ]:
tokyo.coordinates

(35.689722, 139.691667)

In [ ]:
tokyo[1]

'JP'

In [ ]:
# Example 5-5. Named tuple attributes and methods (continued from the previous example)

City._fields # ._fields is a tuple with the field names of the class.

('name', 'country', 'population', 'coordinates')

In [ ]:
Coordinate = namedtuple('Coordinate', 'lat lon')

delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data) # ._make() builds City from an iterable; City(*delhi_data) would do the same.
delhi._asdict() # ._asdict() returns a dict built from the named tuple instance.

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

In [ ]:
import json
json.dumps(delhi._asdict()) # ._asdict() is useful to serialize the data in JSON format, for example.

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

In [ ]:
# Example 5-6. Named tuple attributes and methods, continued from Example 5-5

Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=[23, 'WGS84'])
Coordinate(0,0)

Coordinate(lat=0, lon=0, reference='WGS84')

In [ ]:
Coordinate._field_defaults

{'lon': 23, 'reference': 'WGS84'}

## Typed Named Tuples

- **Definition**:  An extension of the standard `namedtuple` providing type annotations for fields.
  
- **Module**: Part of the `typing` module since Python 3.5.

- **Type Checking**: Provides static type checks when used with tools like `mypy`.
  
- **Immutability**: Instances are immutable, similar to regular `namedtuple`.
  
- **Methods**
  - Inherits methods from `namedtuple` like `_asdict()`, `_fields`, `_replace()`, etc.
  
- **Custom Methods**
  - Supports adding custom methods, which is not possible with regular `namedtuple`.
  
- **Metaclass Functionality**: Uses metaclasses for class creation, making it seem like a superclass but it's not.
  
- **Limitations**: Doesn't enforce type constraints at runtime. Incorrect types will be stored without error.
  


In [ ]:
# The Coordinate class with a default field from Example 5-6 can be written using typing.NamedTuple
from typing import NamedTuple

class Coordinate(NamedTuple):
  lat: float # Every instance field must be annotated with a type.
  lon: float
  reference: str = 'WGS84' # The reference instance field is annotated with a type and a default value.

## Type Hints 101

- Type hints, introduced in PEP 484, allow type annotations in Python.
  
- **Interpreter Behavior**: Not enforced at runtime by the Python interpreter.
      
- **Static Type Checkers**: Tools like `mypy` check code for type consistency based on hints.
  
- **Optional Types**: Use `Optional` from `typing` for values that can be `None`.
  
- **Complex Types**: The `typing` module offers types like `List`, `Dict`, `Tuple`, etc.
- **Generics**: Allow type parameters for type-safe classes/functions.
  
- **Class & Variable Annotations**: Type hints aren't limited to function signatures.
  
- **Runtime Type Checking**: Possible with tools like `typeguard`, though not native to type hints.


### No Runtime Effect

In [ ]:
# Example 5-9. Python does not enforce type hints at runtime
import typing

class Coordinate(typing.NamedTuple):
    lat: float
    lon: float # instance must be of type float


In [ ]:
trash = Coordinate('Ni!', None) # the assignment to trash uses a str and None
trash

Coordinate(lat='Ni!', lon=None)

In [ ]:
Coordinate(lat='Ni!', lon=None) # no type checking at runtime!

Coordinate(lat='Ni!', lon=None)

* The type hints are intended primarily to support third-party type checkers, like Mypy or the PyCharm IDE built-in type checker.
* they check Python source code “at rest,” not running code.

### Variable Annotation Syntax

* A concrete class, for example, str or FrenchDeck
* A parameterized collection type, like list[int], tuple[str, float], etc.
* typing.Optional, for example, Optional[str]—to declare a field that can be a str or None


**NOTE**: The classes built by `typing.NamedTuple` and `@dataclass` have an `__annotations__` attribute holding the type hints for the fields. However, reading from `__annotations__` directly is not recommended. Instead, the recommended best practice to get that information is to call `inspect.get_annotations(MyClass)` (added in Python 3.10) or `typing.get_type_hints(MyClass)` (Python 3.5 to 3.9).

### The Meaning of Variable Annotations

In [ ]:
# Example 5-10. meaning/demo_plain.py: a plain class with type hints
class DemoPlainClass:
  a: int            # entry in __annotations__ , no attribute named a is created in the class.
  b: float = 1.1     # saved as an annotation , a class attribute with value 1.1.
  c = 'spam'        # just a plain old class attribute, not an annotatio

In [ ]:
DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [ ]:
DemoPlainClass.a

AttributeError: ignored

In [ ]:
DemoPlainClass.b

1.1

In [ ]:
DemoPlainClass.c

'spam'

In [ ]:
o = DemoPlainClass() #example object
o.a # AttributeError: 'DemoPlainClass' object has no attribute 'a'


AttributeError: ignored

In [ ]:
print(o.b)
print(o.c)

1.1
spam


#### Inspecting a typing.NamedTuple

In [ ]:
# Example 5-11. meaning/demo_nt.py: a class built with typing.NamedTuple
import typing

class DemoNTClass(typing.NamedTuple):
  a: int               # an annotation and also an instance attribute.
  b: float = 1.1        # another annotation and also an instance attribute.
  c = 'spam'           # just a plain old class attribute; no annotation will refer to it.

In [ ]:
DemoNTClass.__annotations__

{'a': int, 'b': float}

In [ ]:
DemoNTClass.a  # The a and b class attributes are descriptors

_tuplegetter(0, 'Alias for field number 0')

In [ ]:
DemoNTClass.b

_tuplegetter(1, 'Alias for field number 1')

In [ ]:
DemoNTClass.c

'spam'

In [ ]:
# DemoNTClass also gets a custom docstring
DemoNTClass.__doc__

'DemoNTClass(a, b)'

In [ ]:
nt = DemoNTClass(8)
print(nt)
print(nt.a)
print(nt.b)
print(nt.c)

DemoNTClass(a=8, b=1.1)
8
1.1
spam


#### Inspecting a class decorated with dataclass

In [ ]:
# Example 5-12. meaning/demo_dc.py: a class decorated with @dataclass from dataclasses import dataclass

@dataclass
class DemoDataClass:
    a: int         # an annotation and also an instance attribute controlled by a descriptor.
    b: float = 1.1  # an annotation, and also becomes an instance attribute with a descriptor and a default value 1.1
    c = 'spam'     # c is just a plain old class attribute; no annotation will refer to it.

In [ ]:
DemoDataClass.__annotations__

{'a': int, 'b': float}

In [ ]:
DemoDataClass.__doc__

'DemoDataClass(a: int, b: float = 1.1)'

In [ ]:
DemoDataClass.a  # a attribute only exist in instances of DemoDataClass
# It will be a public attribute that we can get and set, unless the class is frozen.

AttributeError: ignored

In [ ]:
DemoDataClass.b , DemoDataClass.c

(1.1, 'spam')

In [ ]:
dc = DemoDataClass(9)  # a and b are instance attributes, and c is a class attribute we get via the instance.
print(dc.a)
print(dc.b)
print(dc.c)

9
1.1
spam


In [ ]:
# DemoDataClass instances are mutable—and no type checking is done at runtime
dc.a = 10
dc.b = 'oops'
dc.c = 'whatever'

# More About @dataclass

## Default implementation of magic methods


The `@dataclass` decorator will add the equivalent of these methods to the `InventoryItem` class:

In [ ]:
@dataclass
class InventoryItem:
    '''Class for keeping track of an item in inventory.'''
    name: str
    unit_price: float
    quantity_on_hand: int = 0

def __init__(self, name: str, unit_price: float, quantity_on_hand: int = 0) -> None:
    self.name = name
    self.unit_price = unit_price
    self.quantity_on_hand = quantity_on_hand

def __repr__(self):
    return f'InventoryItem(name={self.name!r}, unit_price={self.unit_price!r}, quantity_on_hand={self.quantity_on_hand!r})'

def __eq__(self, other):
    if other.__class__ is self.__class__:
        return (self.name, self.unit_price, self.quantity_on_hand) == (other.name, other.unit_price, other.quantity_on_hand)
    return NotImplemented

def __ne__(self, other):
    if other.__class__ is self.__class__:
        return (self.name, self.unit_price, self.quantity_on_hand) != (other.name, other.unit_price, other.quantity_on_hand)
    return NotImplemented

`@dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)`

* The * in the first position means the remaining parameters are keyword-only.
* frozen=True: Protects against accidental changes to the class instances.
* order=True: Allows sorting of instances of the data class.

| Option       | Meaning                                              | Default | Notes                                                                                          |
|--------------|------------------------------------------------------|---------|------------------------------------------------------------------------------------------------|
| `init`       | Generate `__init__`                                  | True    | Ignored if `__init__` is implemented by user.                                                  |
| `repr`       | Generate `__repr__`                                  | True    | Ignored if `__repr__` is implemented by user.                                                  |
| `eq`         | Generate `__eq__`                                    | True    | Ignored if `__eq__` is implemented by user.                                                    |
| `order`      | Generate `__lt__`, `__le__`, `__gt__`, `__ge__`      | False   | If True, raises exceptions if eq=False, or if any of the comparison methods are defined or inherited. |
| `unsafe_hash`| Generate `__hash__`                                  | False   | Complex semantics and several caveats— see: [dataclass documentation](link_to_docs).                |
| `frozen`     | Make instances “immutable”                                       | False   | Instances will be reasonably safe from accidental change, but not really immutable.             |



## Field Options

- A **field** in a `@dataclass` is defined as a class variable that has a _type annotation_.
- The **order** of the fields in all of the generated methods is the order in which they appear in the class definition.
- Parameters which can be set to customize field properties:
    - `default`: Provides a default value for the field.
    - `default_factory`: A function that returns the initial value of the field.
    - `init`: If true (the default), this field is included as a parameter to the generated `__init__` method.
    - `repr`: If true (the default), this field is included in the string returned by the generated `__repr__` method.
    - `compare`: If true (the default), this field is included in the generated equality and comparison methods (`__eq__`, `__gt__`, etc.).
    - `hash`: Can be `True`, `False`, or `None`. If `None` (the default), uses the value of `compare`: if compare is true, then hash is true.
    - `metadata`: A mapping (dictionary) which can contain user-defined data.
- **Init-only Fields**: These are fields that can only be set during the initialization of the object. This is useful when we want to pass arguments to `__init__` that are not instance fields.
- **Mutable Default Fields**: These are fields that have a default value that is mutable.
- **Field List**: You can get a list of fields using `dataclasses.fields()` which returns a list of field objects.


|Option|Meaning|Default|
|------|-------|-------|
|default|Default value for field|_MISSING_TYPE|
|default_factory|0-Parameter function used to produce a default|_MISSING_TYPE|
|init|Include field in parameters to `__init__`|True|
|repr|Include field in `__repr__`|True|
|compare|Use field in comparison methods `__eq__`, `__lt__`, etc.|True|
|hash|Include field in `__hash__` calculation|None|
|metadata|Mapping with user-defined data; ignored by the `@dataclass`|None|


### Mutable Default

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guests: list[str] = field(default_factory=list)  # To see error write `guests: list = []`
    #guests: list[str] = []

*NOTE*: `@dataclass` default factory is a partial solution to solve shared mutable default values that only applies to **list**, **dict**, and **set**. Thus, other mutable values pass this dataclass rejection without error.

*NOTE*: `list[str]` means _a list of string elements._ However, `list` shows _guests_ accept a list of elements which have no type constraint.

In [ ]:
@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)
    athlete: bool = field(default=False, repr=False)

club_member = ClubMember('Alex')

print(club_member)
print(f"{club_member.athlete=}")

ClubMember(name='Alex', guests=[])
club_member.athlete=False


**NOTE**: The default option exists because the field call takes the place of the default value in the field annotation.

## Post-init Processing

One of the key features of dataclasses is the `__post_init__()` method. When defined on the class, it will be called by the generated `__init__()`, normally as `self.__post_init__()`. This function is called by the built-in `__init__()` after initialization of all the attributes of a DataClass.

This feature is handy when certain attributes are _dependent_ on the parameters passed in the `__init__()` but do not get their values directly from them. They get their values after performing some operation, such as _validation_ or _computation_, on a subset of arguments received in the constructor.

In [ ]:
# tag::DOCTESTS[]
"""
``HackerClubMember`` objects accept an optional ``handle`` argument::

    >>> anna = HackerClubMember('Anna Ravenscroft', handle='AnnaRaven')
    >>> anna
    HackerClubMember(name='Anna Ravenscroft', guests=[], handle='AnnaRaven')

If ``handle`` is omitted, it's set to the first part of the member's name::

    >>> leo = HackerClubMember('Leo Rochael')
    >>> leo
    HackerClubMember(name='Leo Rochael', guests=[], handle='Leo')

Members must have a unique handle. The following ``leo2`` will not be created,
because its ``handle`` would be 'Leo', which was taken by ``leo``::

    >>> leo2 = HackerClubMember('Leo DaVinci')
    Traceback (most recent call last):
      ...
    ValueError: handle 'Leo' already exists.

To fix, ``leo2`` must be created with an explicit ``handle``::

    >>> leo2 = HackerClubMember('Leo DaVinci', handle='Neo')
    >>> leo2
    HackerClubMember(name='Leo DaVinci', guests=[], handle='Neo')
"""
# end::DOCTESTS[]

# tag::HACKERCLUB[]
from dataclasses import dataclass

@dataclass
class HackerClubMember(ClubMember):
    all_handles = set()
    handle: str = ''

    def __post_init__(self):
        cls = self.__class__
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:
            msg = f'handle {self.handle!r} already exists.'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)
# end::HACKERCLUB[]

In [ ]:
HackerClubMember.__doc__

"HackerClubMember(name: str, guests: list = <factory>, athlete: bool = False, handle: str = '')"

**NOTE**: `<factory>` is a short way of saying that some callable will produce the default value for guests

## Typed Class Attributes

To add type annotation to class attributes if we use ordinary type annotations like `set[str]` the `@dataclass` will change it to an instance attribute. To prevent that behavior we use `typing.ClassVar[<attribute type>]`.

In [ ]:
from typing import ClassVar
from dataclasses import field

@dataclass
class Foo:
    ins_attr: set[str] = field(default_factory=set)
    cls_attr: ClassVar[set[str]] = set()

print(Foo.__doc__)
print(f"{Foo.cls_attr=}")

Foo(ins_attr: set[str] = <factory>)
Foo.cls_attr=set()


**NOTE**: This is one of the cases that `@dataclass` considers type annotation.

## Initialization Variables That Are Not Fields

- Sometimes you need to pass arguments to `__init__` that are not instance fields. These arguments called **init-only variables**.
- To declare an attribute as _init-only_ we use `dataclasses.InitVar[<attribute type>]`.

In [ ]:
from dataclasses import dataclass, InitVar

class DatabaseType:
    def lookup(self, search_text: str):
        return len(search_text)

@dataclass
class C:
    i: int
    j: int = None
    database: InitVar[DatabaseType] = None

    def __post_init__(self, database):
        if self.j is None and database is not None:
            self.j = database.lookup('j')

my_database = DatabaseType()
c = C(10, database=my_database)

print(f"{c.i=}, {c.j=}")
print(f"{c.database=}")


c.i=10, c.j=1
c.database=None


## Inheritance
Behind the scene

- It looks at all the base classes (the classes that your class inherits from) in reverse order. This order is known as the Method Resolution Order (MRO). It starts from the most basic class, object, and goes up to your class.

- For each base class that is also a Data Class, it takes the fields (the variables defined in the class) and adds them to a list.

- After it has looked at all the base classes, it then adds the fields from your class to the list.

- This list of fields is then used to create methods for your class like `__init__`, `__repr__`, and others.

- If a field is defined in both your class and a base class, the one in your class is used. This is because fields are added to the list in the order they are found, so later fields override earlier ones.

**NOTE**: If a field is excluded from `__init__` (using `init=False`) and the field also specifies default_factory, then the default factory function will always be called from the generated `__init__` function. This happens because there is no other way to give the field an initial value.

In [ ]:
from dataclasses import dataclass

@dataclass
class Base:
    x: float = 11.0
    y: int = 4


@dataclass
class C(Base):
    z: int = 10
    x: int = 15


print(C.x,C.y,C.z)
print(Base.x,Base.y) # 'Base' has no attribute 'z'

15 4 10
11.0 4


## @dataclass Example: Dublin Core Resource Record

In [ ]:
"""
Media resource description class with subset of the Dublin Core fields.

Default field values:

    >>> r = Resource('0')
    >>> r  # doctest: +NORMALIZE_WHITESPACE
    Resource(identifier='0', title='<untitled>', creators=[], date=None,
    type=<ResourceType.BOOK: 1>, description='', language='', subjects=[])

A complete resource record:
# tag::DOCTEST[]

    >>> description = 'Improving the design of existing code'
    >>> book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition',
    ...     ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19),
    ...     ResourceType.BOOK, description, 'EN',
    ...     ['computer programming', 'OOP'])
    >>> book  # doctest: +NORMALIZE_WHITESPACE
    Resource(identifier='978-0-13-475759-9', title='Refactoring, 2nd Edition',
    creators=['Martin Fowler', 'Kent Beck'], date=datetime.date(2018, 11, 19),
    type=<ResourceType.BOOK: 1>, description='Improving the design of existing code',
    language='EN', subjects=['computer programming', 'OOP'])

# end::DOCTEST[]
"""

# tag::DATACLASS[]
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum, auto
from datetime import date


class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()


@dataclass
class Resource:
    """Media resource description."""
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)
# end::DATACLASS[]


from typing import TypedDict


class ResourceDict(TypedDict):
    identifier: str
    title: str
    creators: list[str]
    date: Optional[date]
    type: ResourceType
    description: str
    language: str
    subjects: list[str]


if __name__ == '__main__':
    r = Resource('0')
    description = 'Improving the design of existing code'
    book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition',
                    ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19),
                    ResourceType.BOOK, description,
                    'EN', ['computer programming', 'OOP'])
    print(book)
    book_dict: ResourceDict = {
        'identifier': '978-0-13-475759-9',
        'title': 'Refactoring, 2nd Edition',
        'creators': ['Martin Fowler', 'Kent Beck'],
        'date': date(2018, 11, 19),
        'type': ResourceType.BOOK,
        'description': 'Improving the design of existing code',
        'language': 'EN',
        'subjects': ['computer programming', 'OOP']}
    book2 = Resource(**book_dict)
    print(book == book2)

Resource(identifier='978-0-13-475759-9', title='Refactoring, 2nd Edition', creators=['Martin Fowler', 'Kent Beck'], date=datetime.date(2018, 11, 19), type=<ResourceType.BOOK: 1>, description='Improving the design of existing code', language='EN', subjects=['computer programming', 'OOP'])
True


# Data Class as a Code Smell
Main idea of Object Oriented Programming is to place _behavior_ and _data_ together in the same code unit which be called **class**. Therefore, if a class is wiledly used but has no significant behavior of its own, it is possible that code dealing with its instances is scattered in methods and functions throughout the system.

**NOTE**: _Code Smell_ is a coding pattern that may be a symptom of poor object-oriented design. In other term, patterns in code that may indicate the need for refactoring.

By the way, there are some common scenarios where it makes sense to have a data class with little or no behavior.

## Data Class as Scaffolding
In this scenario, the data class is an initial, simplistic implementation of a class to jump start a new project or module. With time, the class should get its own methods, instead of relying on methods of other classes to operate on its instances. Scaffolding is temporary; eventually your custom class may become fully independent from the builder you used to start it.

## Data Class as Intermediate Representation
A data class can be useful to build records about to be exported to JSON or some other interchange format, or to hold data that was just imported, crossing some system boundary.

**NOTE**: Python’s data class builders all provide a method or function to convert an instance to a plain dict, and you can always invoke the constructor with a dict used as keyword arguments expanded with `**`. Such a dict is very close to a **JSON** record.

**NOTE**: In this scenario, the data class instances should be handled as immutable objects even if the fields are mutable, you should not change them while they are in this intermediate form. If you do, you’re losing the key benefit of having data and behavior close together. When importing/exporting requires changing values, you should implement your own builder methods instead of using the given “as dict” methods or standard constructors.

# Pattern Matching Class Instances
Class patterns are designed to match class instances by type and (optionally) by attributes.

There are three variations of class patterns: **simple**, **keyword**, and **positional**. We’ll study them in that order.

**NOTE**: The subject of a class pattern can be any class instance, not only instances of data classes.

## Simple Class Patterns

the syntax for class pattern is as following:

```python
match x:
    case float():
        do_something_with(x)
```
However, becareful to not use `float` instead of `float()`, because `case float:` matches any subject as python sees `float` as a variable.

**NOTE**: Nine blessed built-in types:
- bytes
- dict
- float
- frozenset
- int
- list
- set
- str
- tuple

**NOTE**: If the class is not one of hose nine blessed built-ins, then the argument-like variables represent patterns to be matched against attributes of an instance of that class.

## Keyword Class Patterns

In [ ]:
import typing

class City(typing.NamedTuple):
    continent: str
    name: str
    country: str

cities = [
    City('Asia', 'Tokyo', 'JP'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('South America', 'São Paulo', 'BR'),
]

In [ ]:
def match_asian_cities():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia'):
                results.append(city)
    return results

match_asian_cities()

[City(continent='Asia', name='Tokyo', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [ ]:
def match_asian_countries():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia', country=cc):
                results.append(cc)
    return results

match_asian_countries()

['JP', 'IN']

## Positional Class Patterns

In [ ]:
def match_asian_cities_pos():
    results = []
    for city in cities:
        match city:
            case City('Asia'):
                results.append(city)
    return results

match_asian_cities_pos()

[City(continent='Asia', name='Tokyo', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [ ]:
def match_asian_countries_pos():
    results = []
    for city in cities:
        match city:
            case City('Asia', _, country):
                results.append(country)
    return results

match_asian_countries_pos()

['JP', 'IN']

In [ ]:
City.__match_args__

('continent', 'name', 'country')

# Why not just use namedtuple or NamedTuple?

## namedtuple

- Any namedtuple can be accidentally compared to any other with the same number of fields or other tuples. For example: `Point3D(2017, 6, 2) == Date(2017, 6, 2)` or `Point2D(1, 10) == (1, 10)`. With Data Classes, this would return `False`.
- Instances are always iterable, which can make it difficult to add fields. If a library defines:

```python
Time = namedtuple('Time', ['hour', 'minute'])
def get_time():
    return Time(12, 0)
```

Then if a user uses this code as:

```python
hour, minute = get_time()
```

then it would not be possible to add a second field to Time without breaking the user’s code.

- No option for mutable instances.
- Cannot specify default values.
- Cannot control which fields are used for `__init__`, `__repr__`, etc.
- Cannot support combining fields by inheritance

## NamedTuple
For classes with statically defined fields, it does support similar syntax to Data Classes, using type annotations. This produces a **namedtuple**, so it shares namedtuples benefits and some of its downsides. Data Classes, unlike `typing.NamedTuple`, support combining fields via inheritance.

# Useful References

- [Write Pythonic and Clean Code With namedtuple](https://realpython.com/python-namedtuple/)
- [Which Python @dataclass is best? Feat. Pydantic, NamedTuple, attrs...](https://www.youtube.com/watch?v=vCLetdhswMg)
- [PEP 526 – Syntax for Variable Annotations](https://peps.python.org/pep-0526/)
- [PEP 484 – Type Hints](https://peps.python.org/pep-0484/)
- [dataclasses — Data Classes](https://docs.python.org/3/library/dataclasses.html)
- [Code Smell](https://martinfowler.com/bliki/CodeSmell.html)
- [PEP 557 - Data Classes](https://peps.python.org/pep-0557/)

# Lecturers

1. Farzin Vatani, [Linkedin](https://www.linkedin.com/in/farzin-vatani)
2. Zohreh Alizadeh, [Linkedin](https://www.linkedin.com/in/zohreh-bayramalizadeh/)


present date : 2023-10-20


# Reviewers

1. Hosein Toodehroosta, review date: 2023-10-19, [LinkedIn](https://www.linkedin.com/in/hossein-toodehroosta-4b34b9191?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=android_app)
2. Mahyar Jahaninasab, review date: 2023-10-19, [Github](https://github.com/mahyar-jahaninasab)
3. Reza Hashemian, review date: 2023-10-20, [LinkedIn](https://www.linkedin.com/in/rezahashemian)
4. Mahya Asgarian, review date: 2023-10-20, [LinkedIn](https://www.linkedin.com/in/mahya-asgarian-9a7b13249)
5. Ghazal Tajik, review date: 2023-10-20, [LinkedIn](https://www.linkedin.com/in/ghazal-tajik-1736a126b)
5. Mehran Faraji, review date: 2023-10-20, [LinkedIn](https://www.linkedin.com/in/mehranfaraji/)